Update script. Gathers the current prices in the specified currencies, then appends them to the pagilla database. Distinct from the test dataset due to a gap in data from the api. Also shares the restriction on currencies for testing purposes.

In [ ]:
#Import modules
import pandas as pd
import requests
import psycopg2
import pandas.io.sql as sqlio

#Initialise update table
#del df_update
now_url = 'https://api.coindesk.com/v1/bpi/currentprice/USD.json'
response4 = requests.get(now_url)
temp = response4.json()
timestamp = [temp['time']['updatedISO'][:10]]
df_update = pd.DataFrame({'date':timestamp})

#Iterate over currencies and add to update table
currency_code = {'USD':'usd','GBP':'gbp','JPY':'jpy','RUB':'rub','AED':'aed'}
for i in range(0,len(currency_code)):
  currency_big = list(currency_code)[i]
  currency_small = list(currency_code.values())[i]
  now_url = f'https://api.coindesk.com/v1/bpi/currentprice/{currency_big}.json'
  response3 = requests.get(now_url)
  update = response3.json()
  price = update['bpi'][f'{currency_big}']['rate']
  df_update[f'{currency_small}'] = price

#Connect to pagila and add new values

conn = psycopg2.connect(host="data-sandbox.c1tykfvfhpit.eu-west-2.rds.amazonaws.com",
                        database = "pagila",
                        user="de8_dobu28",
                        password="JQmhe99+",
                        port=5432,
                        options='-c search_path=student')

cur = conn.cursor()

listabs = df_update.values.tolist()[0]
listabs = [j.replace(',', '') for j in listabs]
listbeta = [listabs[0]]
listabs = [float(i) for i in listabs[1:]]
for i in listabs:
  listbeta.append(i)
listabs = listbeta
query = 'INSERT INTO db_cap_databrick (date, usd, gbp, jpy, rub, aed) VALUES (%s,%s,%s,%s,%s,%s);'
record = (listabs[0],listabs[1],listabs[2],listabs[3],listabs[4],listabs[5])
#diff = df_diff.values.tolist()[0]
cur.execute(query,record)
conn.commit()

cur.close()
conn.close()